# Music21 - part 3

In [ ]:
from music21 import *

### Calculating harmonic intervals

In [ ]:
stream2 = corpus.parse('beethoven/opus18no1/movement1.krn')
stream2.measures(0,16).show()

Let's calculate the vertical intervals between the two violin parts wherever they sound together at the same time.

For all music21 is aware a "chord" is simply any two events that happen at the same time. So a vertical interval will just be a chord with 2 notes in it. We can therefore use "chordify" to get vertical intervals.

Let's try it out on a few measures at a time:

In [ ]:
#create new score object and append the parts you want into it.
vlns = stream.Score()
vlns.append([stream2.parts[0], stream2.parts[1]])

In [ ]:
vlns.measures(5,8).show()

Aside --> By the way: This tool is a bit buggy, but just becuase I haven't remembered to show it to you, there's also the option of midi playback within the notebook, which may be useful for some of you:

In [ ]:
vlns.measures(5,8).show('midi')

Anyway, back to getting vertical intervals:

In [ ]:
ints = vlns.measures(5,8).chordify()
ints.show()

Remember that the output of chordify is a new stream.part object, so to get the intervals (i.e., chords) we still have to use .recurse()

In [ ]:
#recall that .notes will get all single note and chord objects
for i in ints.recurse().notes:
    print(i.commonName)

In [ ]:
#let's have a look at one chord object
x = ints.recurse().notes[0]

In [ ]:
x.notes

In [ ]:
x.pitches

In [ ]:
interval.Interval(x.notes[0],x.notes[1]).diatonic.directedSemiSimpleName

Although a bit annoying, a chord object doesn't have a "semitones" attribute since it can contain more than 2 things. Therefore, you can either call the "commonName" attribute, or else you can expand the chord to get the notes inside it, and pass the first and second items in the tuple to get the interval size.

### Exercise #1:

Examine the distribution of harmonic intervals between the first two violins in Mozart's "mozart/k458/movement1.mxl" in the built-in corpus.

See if you can plot it either with music21's plotting abilities, or using the pandas library.

**hint:** you can make a pandas.Series object by passing a list (as shown in part2)

## Some tools for examining similarity

In [ ]:
lassie = corpus.parse('ryansMammoth/BlackEyedLassieReelThe.abc')
lassie.show()

Music21 has a `repeat` module that has some built-in tools for finding repeated and similar measures. Be careful in dealing with pickup measures.

Note that in the above you should ignore the measure numbers printed out, since the .abc file counts the pickup from 1 and music21 starts at 0.

Also the single note in "measure 10" is messing things up. We can manually intervene:

In [ ]:
#make a copy in case we have to revert
import copy
newlassie = copy.deepcopy(lassie)

In [ ]:
#get rid of the barline in m.8
newlassie.parts[0].measure(8).removeByClass('Barline')

In [ ]:
#get rid of measure 9 and contents
m9 = newlassie.parts[0].measure(9)
newlassie.parts[0].remove(m9, recurse=True)

#add the B- back in
newlassie.parts[0].measure(8).insert(3.5, note.Note('B-4', quarterLength=0.5))

In [ ]:
newlassie.show()

In [ ]:
#show measures or groups of measures that repeat (this will get largest groups)
from music21 import repeat
repeat.RepeatFinder(newlassie.parts[0]).getSimilarMeasureGroups()

In [ ]:
#This is a thing... but I have no idea how to interpret the results. If anyone can figure it out, bonus points!
# http://web.mit.edu/music21/doc/moduleReference/moduleRepeat.html?highlight=repeatfinder%20getmeasuresimilaritylist#music21.repeat.RepeatFinder.getMeasureSimilarityList
repeat.RepeatFinder(newlassie.parts[0]).getMeasureSimilarityList()

The "pickup" finder may be useful:

In [ ]:
repeat.RepeatFinder(lassie.parts[0]).getQuarterLengthOfPickupMeasure()

We could also come up with a custom routine to do something like this ourselves, but with more control over the object we are computing the similarity on (e.g., pitch class content, duration sequence, beat strength, etc.)

Here is a piece of coe that will create lists of four measures (a phrase) and place them inside a larger list (streamLists) which should be a container for phrases.

In [ ]:
streamLists = []
fourms = []
for i, m in enumerate(newlassie.parts[0].getElementsByClass('Measure')):
    #append each measure to the inner list
    # note that 0 modulo 4 is still 0, so...
    if i != 0:
        fourms.append(m)
    #whenever the length of the list reaches 4 items place in outer list, and reset inner list to empty
    if (i % 4 == 0) and (i != 0 ): 
        streamLists.append(fourms)
        fourms = []
    #in case the entire piece length in measures is not divisible by four, grab remaining measures:
    elif i == len(newlassie.parts[0].getElementsByClass('Measure')):  
        streamLists.append(fourms)
      

streamLists  

Now that I have a list of phrases, I can iterate over all the notes in each phrase, and return a list of some feature of interest in order to compute the similarity. Let's look at durations, for example:

In [ ]:
#create empty list
durs = []
#iterate over list of phrases:
for l in streamLists:
    #create inner list to maintain structure
    n_list = []
    #get all the notes at the phrase level, return or compute feature of interest
    for m in l:
        for n in m.notes:
            n_list.append(n.duration.quarterLength)
    durs.append(n_list)
        
durs

So I have a list of durations for each phrase. How might I be able to calculate the similarity for pairs of phrases? Let's look at two useful tools from two non-music21 libraries `itertools.combinations` and `difflib.SequenceMatcher`:

In [ ]:
#demo how itertools.combinations works
l = [[1,2],[3,4],[5,6]]
import itertools
x = itertools.combinations(l,2)
print([i for i in x])

The following tool is also useful:
https://docs.python.org/3/library/difflib.html#difflib.SequenceMatcher

In [ ]:
#demo of how "sequence matcher" works
import difflib
s1=[1,8,3,9,4,9,3,8,1,2,3]
s2=[1,8,1,3,9,4,9,3,8,1,2,3]
sm=difflib.SequenceMatcher(None,s1,s2)
sm.ratio()

In [ ]:
#demo 2
s1=['a','b','c','d']
s2=['a','b','c','e']
sm=difflib.SequenceMatcher(None,s1,s2)
sm.ratio()

In [ ]:
#here is a piece of code that will compute the correlation across every pair of phrases in this group of 4 phrases
#get pair-wise combinations of all phrases
pairs = list(itertools.combinations(durs, 2))
#to keep track of pairs, create list of names in same order. Note I'm using numbers but you could easily name them.
pairnames = list(itertools.combinations(list(range(0,len(durs))),2))

for i, j in enumerate(pairs):
    sm = difflib.SequenceMatcher(None, j[0], j[1])
    print(pairnames[i], sm.ratio())

Note that `itertools.combinations` returns an interable which is not hashable. If you want to index the combinations or subsets of combinations, you have to first convert it to a list as I have done above.

### Exercise 2

Upload the Beatle's tune "Eight Days a Week" from Canvas to the server and import it into music21.
Extract the upper-most part.

    (a) compute the similarity of measures using music21's `RepeatFinder` tools (as shown above). 
    
    (b) compute the similarity of the similarity of the rhythms, using any tools or functions you like
    
    (c) compute the similarity of the pitch class content, using any tools or functions you like.